In [947]:
import urllib.request as request
import datetime
import pandas as pd
from ratelimiter import RateLimiter
import math

In [ ]:
#need to ensure this reads the old data and checks online for new recipes, stamping them with a date of import.

# create storage

In [667]:
#df = pd.DataFrame() #STORES LINKS TO RECIPES
#store_recipes = {} #STORES RECIPES

#IF SCRAPING, DO NOT SUBMIT THE ABOVE AS YOU MAY LOSE EVERYTHING
#THE SCRAPER ONLY SAVES TO CSV EVERY 50 RECIPES 

# Scrape for recipe urls

In [263]:
#GET RECIPE NAMES AND LINKS FROM JamieOliver.Com
site = {}
domain = 'https://www.jamieoliver.com'
food_cats = ['chicken','pasta','vegetables','fish','beef','eggs']

#We are going to scrap each category page to retrieve recipe urls, 
#     which will then be scraped individually to get the ingredients
cat_links = [domain+'/recipes/'+x+'-recipes/' for x in food_cats]
recipe_links = {}
fullhtmls = {}

#each link looks like 'https://www.jamieoliver.com/recipes/CATEGORYNAME-recipes'
#SCRAP SITE
for cat in cat_links:
    recipe_links[cat] = {}
 #   site["categories"] = 
    resp = request.urlopen(cat)
    data = resp.read()
    html = data.decode('utf-8')
    fullhtmls[cat] = html
    #divide the webpage into html tags as a python list
    split_data = html.split('<')
    num_recipes = int(list(filter(lambda k: 'span class="recipe-count"' in k, split_data))[0].split('(')[1][:-1])
    recipe_links[cat]["_totalrecipes"] = num_recipes
    recipe_links[cat]["recipe_names"] = {}
    partial_recipelinks = [x.split('href="')[1].split('"')[0] for x in list(filter(lambda k: 'id="gtm_recipe_subcat_' in k, split_data))]
    recipe_names = [x.split('>')[1] for x in list(filter(lambda k: 'div class="recipe-title">'in k, split_data))]
    for recipe in range(0,len(partial_recipelinks)):
        recipe_links[cat]["recipe_names"][recipe_names[recipe]] = domain+partial_recipelinks[recipe]

# Output recipe names to a df / csv

In [309]:
#Convert dictionary data into csv for easy pandas reading
for cat in recipe_links:
    tmp = pd.DataFrame()
    tmp["recipe_name"] = [x for x in recipe_links[cat]['recipe_names']]
    tmp["recipe_link"] = [recipe_links[cat]['recipe_names'][x] for x in recipe_links[cat]['recipe_names']]
    tmp["category"] = cat.split('/')[-2].split('-')[0]
    #print(pd.DataFrame({x:recipe_links[cat]["recipe_names"][x] for x in recipe_links[cat]["recipe_names"]},index=index))
    if df.empty:
        df = tmp
    else:
        df = pd.concat([df,tmp])
df.to_csv('recipe_list.csv')  

# scrape recipes

In [949]:
#GET SCRAPING

def scraper():
    def scrape_unlimited(urllink):
        #MAKE REQUEST
        return request.urlopen(urllink)
    
    def waiting_fnc():
            @RateLimiter(max_calls=1, period=120)
            def timedelay():
                return
            print(f"\n{datetime.datetime.now().strftime('%H:%M:%S')} HTTP Error 403: Forbidden-\nRate Limiting (retrying in 2 min)")
            print('\n -- -- -- -- -- ')
            timedelay()
            timedelay() #By applying the waiting function twice, it will have to wait 2 min before proceeding
            
    try:
        #MAKE AND READ REQUEST
        resp = scrape_unlimited(urls)
        return resp
    
    except urllib.error.HTTPError:
        
        waiting_fnc()
        resp = scraper()
        return resp

def cleaningfnc(y):
    return y[[str(y).find(x) for x in str(y) if x!=' '][0]:-str(y)[::-1].find([x for x in str(y) if x!=' '][-1])]

for urls in df['recipe_link'].values:
    if df[df['recipe_link']==urls]['recipe_name'].values[0] in store_recipes.keys():
        print(f'Data already available for {urls.split("/")[-2]}.')
    else:
        resp = scraper()
              
        print(f'request {df[df["recipe_link"]==urls].index.values[0]+1} made--')
        data = resp.read()
        html = data.decode('utf-8')
        
        #CONVERT HTML TO USEABLE DATA
        
        #GET DIETARY REQUIREMENTS LABELS
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting dietary requirements..")
        if html.find("class='special-diets-list")!=-1:
            dietary_req = list(filter(lambda k: "span class='full-name'" in k, html.split('class="special-diets-list')[1].split('<')))
            dietary_req = [x.split('>')[1] for x in dietary_req]
        else:
            dietary_req = "None"
        
        #GET NUMBER OF PORTIONS
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting number of portions..")
        if html.find("recipe-detail serves")!=-1:
            num_people = list(filter(lambda k: "recipe-detail serves" in k, html.split('/div')))[0] \
                  .split('/span')[1].replace(' ','').replace('>','').replace('<','')
        else:
            num_people = "NotAvailable"
        
        #GET COOKING TIME      
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting cooking time")
        if html.find("Cooks In</span>")!=-1:
            time = list(filter(lambda k: "recipe-detail time" in k, html.split('/div')))[0] \
                .split('Cooks In</span>')[1].split(' (')[0].replace(' ','').replace('>','').replace('<','')
        else:
            time = "NotAvailable"
        
        #GET INGREDIENTS
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting ingredients..")
        if html.find("Ingredients</h2>")!=-1:
            ingr_mess = [x.split('</li>')[0] for x in list(filter(lambda k: '</li>' in k, html.split('Ingredients</h2>')[1].split('ul>')[1].split('<li>')))]
            
            print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- applying cleaning fnc to ingredients list..")
        #    ingr_mess = [cleaningfnc(x) for x in ingr_mess]
        #    cleaned_ingr_list = []
            print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- further cleaning..")
            clean_ingr = clean_unit = []
            for x in ingr_mess:
                if str(x).replace('  ','')==str(x):
                    clean_ingr.append(x.replace('  ',''))
                    clean_unit.append('single_serving')
                else:
                    clean_ingr.append(str(x)[-str(x)[::-1].find('  '):])
                    clean_unit.append(str(x)[:-str(x)[::-1].find('  ')].replace('  ',''))
        else:
            clean_ingr = clean_unit = "NotAvailable"
        
        #GET NUTRITIONAL VALUE
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting nutritional information..")
        if html.find("nutrition-item-desc")!=-1:
            nutrition_keys = [x.split('\n')[1].replace(' ','') for x in list(filter(lambda k : 'nutrition-item-desc' in k, html.split('<')))]
            nutrition_values = [x.split('\n')[1].replace(' ','') for x in list(filter(lambda k : 'nutrition-item-top' in k, html.split('<')))]
            nutrition = {k:v for k,v in zip(nutrition_keys,nutrition_values)}
        else:
            nutrition = {"NotAvailable":"NotAvailable"}
        
        #GET THE COOKING METHOD
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- getting cooking method..")
        if html.find("<h2>Method</h2>\n")!=-1:
            method = html.split('<h2>Method</h2>\n')[1].split('/div')[0].replace('  ','')
        else: method = "NotAvailable"
        
        #STORING RESULTS IN DICTIONARY
        print(' ',datetime.datetime.now().strftime('%H:%M:%S'),"-- storing results in dictionary..")
        store_recipes[df[df['recipe_link']==urls]['recipe_name'].values[0]] = {"portions":num_people,
                                                                                 "dietary_req":dietary_req,
                                                                                 "cooking_time":time,
                                                                                 "ingredients":{k:v for k,v in zip(clean_ingr,clean_unit)},
                                                                                 "nutrition":nutrition,
                                                                                 "method":method}
        still_to_go = len(df)-df[df["recipe_link"]==urls].index.values[0]-1
        time_left = (still_to_go/218)
        hrs_left = math.floor(time_left)
        mins_left = round((time_left - hrs_left)*60)
        
        print(f'\n-- Scraping successful! --\n   {still_to_go} remaining... (approx {hrs_left}hrs {mins_left}mins)\n')
        if len(store_recipes)%50==0:
            timeofsave = datetime.datetime.now().strftime('%H:%M:%S')
            df2 = pd.DataFrame.from_dict(store_recipes,orient='index')
            df2.to_csv(f'{timeofsave}_({len(store_recipes)} of {len(df)})_recipe_data.csv')
            print("\n\n- SAVING THE LATEST 50 ENTRIES - \n\n")
    #Rate limited
    pass
          
df2 = pd.DataFrame.from_dict(store_recipes,orient='index')
df2.to_csv('recipe_data.csv')
              
#PRINT A FINAL MESSAGE
success_print =  " ___ _   _  ___ ___ ___  ___ ___ \n/ __| | | |/ __/ __/ _ \/ __/ __|  |\n\__ \ |_| | (_| (_|  __/\__ \__ \  |\n|___/\__,_|\___\___\___||___/___/  .\n"   
print(success_print) 
print('----------\n----------\n\nSaved!\n\nFind data in \"recipe_data.csv\"\n-------\n-------')    
          

Data already available for korean-style-roast-chicken.
Data already available for chicken-tikka-skewers.
Data already available for my-perfect-roast-chicken.
Data already available for salina-chicken.
Data already available for chicken-katsu-curry.
Data already available for game-on-curry.
Data already available for craig-david-s-grenadian-baked-chicken.
Data already available for thai-green-chicken-curry-with-aubergines.
Data already available for roasted-game-bird-pie.
Data already available for ultimate-roast-chicken-caesar-salad.
Data already available for flaky-pastry-pesto-chicken.
Data already available for sweet-chicken-surprise.
Data already available for gnarly-peanut-chicken.
Data already available for quick-and-easy-chicken-noodle-stir-fry.
Data already available for sticky-kickin-wings.
Data already available for thai-red-chicken-soup.
Data already available for clare-s-chicken-fajitas-with-cheat-s-mole-sauce.
Data already available for roast-tikka-chicken.
Data already av

Data already available for jamie-s-classic-family-lasagne.
Data already available for pasta-pesto.
Data already available for lobster-mac-n-cheese.
Data already available for royal-pasta-dough.
Data already available for vegan-mac-n-cheese.
Data already available for basil-gnudi.
Data already available for wholewheat-ravioli-with-courgette-flowers-ricotta.
Data already available for rigatoni-with-roasted-tomatoes-ricotta-salata.
Data already available for spinach-ricotta-cannelloni.
Data already available for crab-rigatoni.
Data already available for gluten-free-pasta-dough.
Data already available for carbonara-of-smoked-mackerel.
Data already available for classic-tomato-spaghetti.
Data already available for sienna-miller-s-duck-rag.
Data already available for jools-sweet-pea-prawn-pasta-shells.
Data already available for grandad-s-mussel-linguine-linguine-con-cozze-di-nonno.
Data already available for gennaro-s-spaghetti-alla-puttanesca.
Data already available for southend-stylee-pap

Data already available for michela-s-cauliflower-apple-sweet-potato.
Data already available for michela-s-blueberries-chickpeas-no-cook.
Data already available for michela-s-beetroot-almond-ricotta-no-cook.
Data already available for michela-s-beans-sweetcorn-spinach-no-cook.
Data already available for michela-s-banana-kale-no-cook.
Data already available for kerryann-s-hidden-vegetable-pasta-sauce.
Data already available for helen-s-african-sweet-potato-stew.
Data already available for helen-s-chickpea-spiced-spinach-smash-with-sweet-potato.
Data already available for helen-s-avocado-peas-with-mashed-potato.
Data already available for helen-s-butternut-squash-coconut-stew.
Data already available for vegetable-noodle-stir-fry.
Data already available for simple-chopped-salad.
Data already available for michela-s-easy-minestrone.
Data already available for jools-wholesome-veg-and-bean-soup.
Data already available for gennaro-s-mini-pizzas-with-hidden-veg-sauce.
Data already available for

request 631 made--
  15:13:32 -- getting dietary requirements..
  15:13:32 -- getting number of portions..
  15:13:32 -- getting cooking time
  15:13:32 -- getting ingredients..
  15:13:32 -- applying cleaning fnc to ingredients list..
  15:13:32 -- further cleaning..
  15:13:32 -- getting nutritional information..
  15:13:32 -- getting cooking method..
  15:13:32 -- storing results in dictionary..

-- Scraping successful! --
   490 remaining... (approx 2hrs 15.0mins)

request 632 made--
  15:13:32 -- getting dietary requirements..
  15:13:32 -- getting number of portions..
  15:13:32 -- getting cooking time
  15:13:32 -- getting ingredients..
  15:13:32 -- applying cleaning fnc to ingredients list..
  15:13:32 -- further cleaning..
  15:13:32 -- getting nutritional information..
  15:13:32 -- getting cooking method..
  15:13:32 -- storing results in dictionary..

-- Scraping successful! --
   489 remaining... (approx 2hrs 15.0mins)

request 633 made--
  15:13:33 -- getting dietary req

request 649 made--
  15:13:37 -- getting dietary requirements..
  15:13:37 -- getting number of portions..
  15:13:37 -- getting cooking time
  15:13:37 -- getting ingredients..
  15:13:37 -- applying cleaning fnc to ingredients list..
  15:13:37 -- further cleaning..
  15:13:37 -- getting nutritional information..
  15:13:37 -- getting cooking method..
  15:13:37 -- storing results in dictionary..

-- Scraping successful! --
   472 remaining... (approx 2hrs 10.0mins)

request 650 made--
  15:13:38 -- getting dietary requirements..
  15:13:38 -- getting number of portions..
  15:13:38 -- getting cooking time
  15:13:38 -- getting ingredients..
  15:13:38 -- applying cleaning fnc to ingredients list..
  15:13:38 -- further cleaning..
  15:13:38 -- getting nutritional information..
  15:13:38 -- getting cooking method..
  15:13:38 -- storing results in dictionary..

-- Scraping successful! --
   471 remaining... (approx 2hrs 10.0mins)

request 651 made--
  15:13:38 -- getting dietary req

...

request 1050 made--
  17:04:03 -- getting dietary requirements..
  17:04:03 -- getting number of portions..
  17:04:03 -- getting cooking time
  17:04:03 -- getting ingredients..
  17:04:03 -- applying cleaning fnc to ingredients list..
  17:04:03 -- further cleaning..
  17:04:03 -- getting nutritional information..
  17:04:03 -- getting cooking method..
  17:04:03 -- storing results in dictionary..

-- Scraping successful! --
   71 remaining... (approx 0hrs 20.0mins)

request 1051 made--
  17:04:04 -- getting dietary requirements..
  17:04:04 -- getting number of portions..
  17:04:04 -- getting cooking time
  17:04:04 -- getting ingredients..
  17:04:04 -- applying cleaning fnc to ingredients list..
  17:04:04 -- further cleaning..
  17:04:04 -- getting nutritional information..
  17:04:04 -- getting cooking method..
  17:04:04 -- storing results in dictionary..

-- Scraping successful! --
   70 remaining... (approx 0hrs 19.0mins)

request 1052 made--
  17:04:04 -- getting dietary re

request 1067 made--
  17:10:09 -- getting dietary requirements..
  17:10:09 -- getting number of portions..
  17:10:09 -- getting cooking time
  17:10:09 -- getting ingredients..
  17:10:09 -- applying cleaning fnc to ingredients list..
  17:10:09 -- further cleaning..
  17:10:09 -- getting nutritional information..
  17:10:09 -- getting cooking method..
  17:10:09 -- storing results in dictionary..

-- Scraping successful! --
   54 remaining... (approx 0hrs 15.0mins)

request 1068 made--
  17:10:10 -- getting dietary requirements..
  17:10:10 -- getting number of portions..
  17:10:10 -- getting cooking time
  17:10:10 -- getting ingredients..
  17:10:10 -- applying cleaning fnc to ingredients list..
  17:10:10 -- further cleaning..
  17:10:10 -- getting nutritional information..
  17:10:10 -- getting cooking method..
  17:10:10 -- storing results in dictionary..

-- Scraping successful! --
   53 remaining... (approx 0hrs 15.0mins)

request 1069 made--
  17:10:10 -- getting dietary re

request 1086 made--
  17:16:15 -- getting dietary requirements..
  17:16:15 -- getting number of portions..
  17:16:15 -- getting cooking time
  17:16:15 -- getting ingredients..
  17:16:15 -- applying cleaning fnc to ingredients list..
  17:16:15 -- further cleaning..
  17:16:15 -- getting nutritional information..
  17:16:15 -- getting cooking method..
  17:16:15 -- storing results in dictionary..

-- Scraping successful! --
   35 remaining... (approx 0hrs 10.0mins)

request 1087 made--
  17:16:16 -- getting dietary requirements..
  17:16:16 -- getting number of portions..
  17:16:16 -- getting cooking time
  17:16:16 -- getting ingredients..
  17:16:16 -- applying cleaning fnc to ingredients list..
  17:16:16 -- further cleaning..
  17:16:16 -- getting nutritional information..
  17:16:16 -- getting cooking method..
  17:16:16 -- storing results in dictionary..

-- Scraping successful! --
   34 remaining... (approx 0hrs 9.0mins)

request 1088 made--
  17:16:16 -- getting dietary req

request 1103 made--
  17:22:21 -- getting dietary requirements..
  17:22:21 -- getting number of portions..
  17:22:21 -- getting cooking time
  17:22:21 -- getting ingredients..
  17:22:21 -- applying cleaning fnc to ingredients list..
  17:22:21 -- further cleaning..
  17:22:21 -- getting nutritional information..
  17:22:21 -- getting cooking method..
  17:22:21 -- storing results in dictionary..

-- Scraping successful! --
   18 remaining... (approx 0hrs 5.0mins)

request 1104 made--
  17:22:21 -- getting dietary requirements..
  17:22:21 -- getting number of portions..
  17:22:21 -- getting cooking time
  17:22:21 -- getting ingredients..
  17:22:21 -- applying cleaning fnc to ingredients list..
  17:22:21 -- further cleaning..
  17:22:21 -- getting nutritional information..
  17:22:21 -- getting cooking method..
  17:22:21 -- storing results in dictionary..

-- Scraping successful! --
   17 remaining... (approx 0hrs 5.0mins)

request 1105 made--
  17:22:22 -- getting dietary requ

 ___ _   _  ___ ___ ___  ___ ___ 
/ __| | | |/ __/ __/ _ \/ __/ __|  |
\__ \ |_| | (_| (_|  __/\__ \__ \  |
|___/\__,_|\___\___\___||___/___/  .

----------
----------

Saved!

Find data in "recipe_data.csv"
-------
-------


# Run Scraper - this may take a while! (1 page per 60s)